In [17]:
# requires downloaded xml from https://my.vertabelo.com/drive

from officelib.xllib import Excel
from pywintypes import com_error
import lxml
import lxml.etree as etree
downloads = "C:\\Users\\Nathan\\Downloads"
fp = downloads + "\PBS_System_Property_DB_2019-12-30_13_45.xml"
import clipboard

OOPS = (NameError, com_error)

In [42]:
# creates python code

tree = etree.parse(fp)
root = tree.getroot()
tables = root.find("Tables")
relationships = root.find("References")


class Relationship():
    def __init__(self, pkt=None,pkc=None,fkt=None,fkc=None,name=None):
        self.pkt = pkt
        self.pkc = pkc
        self.fkt = fkt
        self.fkc = fkc
        self.name = name

rels = set()
for ref in relationships:
    pkt = ref.find("PKTable").text.lower()
    fkt = ref.find("FKTable").text.lower()
    
    refcolumns = ref.find("ReferenceColumns")
    
    # is multiple refcol sets possible?
    # reject and figure it out if i ever see it
    if len(refcolumns) > 1:
        raise ValueError("Got multiple reference columns in relationship")
    
    for refcol in refcolumns:
        pkc = refcol.find("PKColumn").text.lower()
        fkc = refcol.find("FKColumn").text.lower()
    
    rel = Relationship(pkt, pkc, fkt, fkc, ref.find("Name").text.lower())
    rels.add(rel)
    
lrels = []
mrels = {}
for r in rels:
    lrels.append(r)
    mrels[(r.fkt, r.fkc)] = r

lines = []
#lines.append("Base = declarative_base()\n\n")

def ctype_to_pytype(ctype):
    if ctype == "integer":
        return "Integer"
    elif ctype == "text":
        return "String"
    else:
        raise ValueError(ctype)

# pass #1 - build (table_id, column_id) -> name map
# required for foreign key specification

id_map = {}
tid_map = {}
for table in tables:
        t_id = table.attrib['Id']
        t_name = table.find("Name").text
        tid_map[t_id] = t_name
        for column in table.find("Columns"):
            c_id = column.attrib['Id']
            c_name = column.find("Name").text
            id_map[(t_id, c_id)] = t_name, c_name        
        
        
# pass #2 - build python code

def class_name(table_name):
    cls_name = table_name
    
    # attempt plural -> singular (may require hand cleanup)
    if cls_name[-1] == 's':
        cls_name = cls_name[:-1]
    return cls_name
        
for table in tables:
    t_id = table.attrib['Id']
    t_name = table.find("Name").text
    cls_name = class_name(t_name)
    t_name = t_name.lower()
    
    pk_ids = set()
    pk_cols = table.find("PrimaryKey").find("Columns")
    for col in pk_cols:
        pk_ids.add(col.text.lower())
        
    lines.append("class %s(Base):"%cls_name)
    lines.append("    __tablename__ = \"%s\""%t_name)
    
    for column in table.find("Columns"):
        c_id = column.attrib["Id"]
        cname = column.find("Name").text.lower()
        ctype = column.find("Type").text.lower()
        nullable = True if column.find("Nullable") == 'true' else False
        
        pk = c_id in pk_ids
        pytype = ctype_to_pytype(ctype)
        
        # check if this column is a foreign key
        r = mrels.get((t_id, c_id))
        if r:
            a,b = id_map[(r.pkt, r.pkc)]
            fk = "ForeignKey('%s.%s')" % (a.lower(),b.lower())
        else:
            fk = ""
            
        # args
        args = [
#             '"%s"' % cname,      # column name
            '%s()' % pytype,     # SQLAlchemy column type
        ]
        
        if fk: args.append(fk)
        
        # keyword args
        kw = {}
        if pk:
            kw['primary_key'] = True
        kw['nullable'] = nullable
        
        line = "    %s = Column(" % cname 
        line += ", ".join(args)
        if kw:
            line += ", " + ", ".join("%s=%s"% kv for kv in kw.items())
        
        line += ")"
        
        # value of 'line' should end up something like this:
        # "    <name> = Column('<name>', <type>(), <FK if foreign key>, <keywords>)
        
        lines.append(line)

        # if this column was a foreign key, add the relationship
        # line as well
#         if fk:
#             name = r.name.replace(t_name+"_","")
#             lines.append("    %s = relationship('%s')" % (name, tid_map[r.pkt]))
                       
        
    # add all of this table's relationships
    # where it is the PK
    spaced = False
    for r in lrels:
        if r.pkt == t_id:
            n = tid_map[r.fkt]
            n2 = class_name(n)
            if not spaced:
                lines.append("")
                spaced = True
            lines.append("    %s = relationship('%s')" % (n.lower(), n2))

    lines.append("\n")
    
code = "\n".join(lines)
print(code)
clipboard.copy(code)

class StringText(Base):
    __tablename__ = "stringtexts"
    id = Column(Integer(), nullable=False, primary_key=True)
    stringtext = Column(String(), nullable=False)

    fieldstrings = relationship('FieldString')


class Field(Base):
    __tablename__ = "fields"
    id = Column(Integer(), nullable=False, primary_key=True)
    fieldname = Column(String(), nullable=False)

    fieldstrings = relationship('FieldString')


class FieldString(Base):
    __tablename__ = "fieldstrings"
    id = Column(Integer(), nullable=False, primary_key=True)
    fields_id = Column(Integer(), ForeignKey('fields.id'), nullable=False)
    stringtexts_id = Column(Integer(), ForeignKey('stringtexts.id'), nullable=False)

    sectionfields = relationship('SectionField')


class Model(Base):
    __tablename__ = "models"
    id = Column(Integer(), nullable=False, primary_key=True)
    partnumber = Column(String(), nullable=False)

    modeldescriptor = relationship('ModelDescriptor')


class ModelDescriptor(Ba

In [ ]:
# creates Excel workbook 

# try:
#     # check to see if object is still connected
#     xl.Visible = xl.Visible 
# except OOPS:
#     xl = Excel()
    
# xl.Visible=True
    
# try:
#     wb.Close(False)
#     del ws
#     del wb
# except OOPS:
#     pass
# finally:
#     wb = xl.Workbooks.Add()
#     while True:
#         try:
#             wb.Worksheets(1).Delete()
#         except com_error:
#             break

# tree = etree.parse(fp)
# root = tree.getroot()
# tables = root.find("Tables")

# fks = []  # (src, dst)

# for table in tables:
#     name = table.find("Name")
#     ws = wb.Worksheets.Add()
#     ws.Name = name.text
#     cell = ws.Cells.Range("A1")
#     print("creating table '%s'"%name.text)
#     for i, column in enumerate(table.find("Columns"),1):
#         col_name = column.find("Name").text
#         cell.Value2 = col_name
#         print("  adding column '%s'"%col_name)
#         if col_name.endswith("_ID"):
#             fks.append((col_name.rsplit("_ID",1)[0],name.text,i))
#         cell=cell.GetOffset(0,1)
        
# try:
#     wb.Worksheets("Sheet1").Delete()
# except com_error:
#     pass